# GROK銘柄 イントラデイ分析

## 目的
- 今日のGROK選定銘柄（Top5/Top10/全銘柄）の前場パフォーマンス分析
- 最適売却時刻の特定（9:00寄付買い → 9:30/10:00/10:30/11:00/11:30売却）

In [19]:
# 必要なライブラリのインポート
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, time

# プロジェクトルートをパスに追加
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from common_cfg.paths import PARQUET_DIR

# Plotly設定
import plotly.io as pio
pio.templates.default = "plotly_white"

print(f"✅ ライブラリ読み込み完了")
print(f"📁 データディレクトリ: {PARQUET_DIR}")

✅ ライブラリ読み込み完了
📁 データディレクトリ: /Users/hiroyukiyamanaka/Desktop/python_stock/dash_plotly/data/parquet


## 1. データ読み込み

In [20]:
# GROK選定銘柄を読み込み
grok_file = PARQUET_DIR / "grok_trending.parquet"
df_grok = pd.read_parquet(grok_file)

print(f"GROK選定銘柄数: {len(df_grok)}")
print(f"選定日時: {df_grok['selected_time'].iloc[0] if 'selected_time' in df_grok.columns else 'N/A'}")
print(f"\nTop5銘柄:")
print(df_grok.nsmallest(5, 'grok_rank')[['ticker', 'stock_name', 'selection_score', 'grok_rank']])

# 分析対象日（今日）
TARGET_DATE = datetime.now().strftime("%Y-%m-%d")
print(f"\n📅 分析対象日: {TARGET_DATE}")

GROK選定銘柄数: 12
選定日時: 23:00

Top5銘柄:
   ticker         stock_name  selection_score  grok_rank
4  3911.T             Aiming            148.0          1
1  4592.T              サンバイオ            160.0          2
8  7078.T          INCLUSIVE             88.0          3
3  4417.T  グローバルセキュリティエキスパート            155.0          4
6  3936.T           グローバルウェイ             90.0          5

📅 分析対象日: 2025-10-27


In [21]:
# 5分足データと日足データを読み込み
prices_5m_file = PARQUET_DIR / "prices_60d_5m.parquet"
prices_1d_file = PARQUET_DIR / "prices_max_1d.parquet"

df_prices_5m = pd.read_parquet(prices_5m_file)
df_prices_1d = pd.read_parquet(prices_1d_file)

print(f"✅ 5分足データ読み込み完了")
print(f"  データ期間: {df_prices_5m['date'].min()} 〜 {df_prices_5m['date'].max()}")
print(f"  銘柄数: {df_prices_5m['ticker'].nunique()}")
print(f"  データ件数: {len(df_prices_5m):,}")

print(f"\n✅ 日足データ読み込み完了")
print(f"  データ期間: {df_prices_1d['date'].min()} 〜 {df_prices_1d['date'].max()}")
print(f"  銘柄数: {df_prices_1d['ticker'].nunique()}")
print(f"  データ件数: {len(df_prices_1d):,}")

✅ 5分足データ読み込み完了
  データ期間: 2025-07-30 09:00:00 〜 2025-10-27 15:25:00
  銘柄数: 68
  データ件数: 272,680

✅ 日足データ読み込み完了
  データ期間: 1999-05-06 00:00:00 〜 2025-10-27 00:00:00
  銘柄数: 68
  データ件数: 450,160


## 2. 当日データの抽出と前処理

In [22]:
# 当日のGROK銘柄のデータを抽出
grok_tickers = df_grok['ticker'].tolist()

# 5分足データ: 当日の売却価格用
df_today_5m = df_prices_5m[
    (df_prices_5m['ticker'].isin(grok_tickers)) &
    (df_prices_5m['date'].dt.date == pd.to_datetime(TARGET_DATE).date())
].copy()

# 日足データ: 当日の始値（寄付価格）用
df_today_1d = df_prices_1d[
    (df_prices_1d['ticker'].isin(grok_tickers)) &
    (df_prices_1d['date'].dt.date == pd.to_datetime(TARGET_DATE).date())
].copy()

print(f"5分足データ（売却価格用）: {len(df_today_5m):,}件")
print(f"  銘柄数: {df_today_5m['ticker'].nunique()} / {len(grok_tickers)}")

print(f"\n日足データ（寄付価格用）: {len(df_today_1d)}件")
print(f"  銘柄数: {df_today_1d['ticker'].nunique()} / {len(grok_tickers)}")

if len(df_today_5m) == 0:
    print(f"\n⚠️ 当日の5分足データがありません。")
if len(df_today_1d) == 0:
    print(f"\n⚠️ 当日の日足データがありません。")

if len(df_today_5m) > 0:
    # 時刻列を追加（5分足）
    df_today_5m['time'] = df_today_5m['date'].dt.time
    df_today_5m['hour_minute'] = df_today_5m['date'].dt.strftime('%H:%M')
    
    print(f"\n5分足時刻範囲: {df_today_5m['hour_minute'].min()} 〜 {df_today_5m['hour_minute'].max()}")
    print(f"\nサンプルデータ（5分足）:")
    print(df_today_5m[['ticker', 'date', 'hour_minute', 'Open', 'Close', 'Volume']].head(5))

if len(df_today_1d) > 0:
    print(f"\nサンプルデータ（日足・始値）:")
    print(df_today_1d[['ticker', 'date', 'Open', 'Close']].head(5))

5分足データ（売却価格用）: 804件
  銘柄数: 12 / 12

日足データ（寄付価格用）: 12件
  銘柄数: 12 / 12

5分足時刻範囲: 09:00 〜 15:25

サンプルデータ（5分足）:
        ticker                date hour_minute    Open   Close   Volume
268124  4880.T 2025-10-27 09:00:00       09:00   528.0   525.0  12700.0
268126  7078.T 2025-10-27 09:00:00       09:00   426.0   431.0   7000.0
268128  3936.T 2025-10-27 09:00:00       09:00   162.0   162.0      0.0
268132  4417.T 2025-10-27 09:00:00       09:00     NaN     NaN      NaN
268134  4011.T 2025-10-27 09:00:00       09:00  3440.0  3470.0      0.0

サンプルデータ（日足・始値）:
        ticker       date    Open   Close
450092  7078.T 2025-10-27   426.0   433.0
450093  4880.T 2025-10-27   528.0   522.0
450096  3936.T 2025-10-27   162.0   162.0
450100  4417.T 2025-10-27  4330.0  4075.0
450101  4011.T 2025-10-27  3440.0  3430.0


## 3. 寄付価格と各時刻での利益計算

In [23]:
# 分析対象時刻の定義
TIME_SLOTS = {
    '09:30': '9:30売却',
    '10:00': '10:00売却',
    '10:30': '10:30売却',
    '11:00': '11:00売却',
    '11:30': '前引け売却',
}

def get_open_price(ticker):
    """日足データから始値（寄付価格）を取得"""
    ticker_data = df_today_1d[df_today_1d['ticker'] == ticker]
    if len(ticker_data) > 0:
        return ticker_data['Open'].iloc[0]
    return None

def get_price_at_time(ticker, target_time):
    """5分足データから指定時刻の価格を取得（前後10分の範囲で最も近い時刻）"""
    df_ticker = df_today_5m[df_today_5m['ticker'] == ticker].copy()
    if len(df_ticker) == 0:
        return None
    
    # 目標時刻の前後10分の範囲でデータを探す
    target_dt = datetime.strptime(target_time, '%H:%M').time()
    df_ticker['time_diff'] = df_ticker['date'].apply(
        lambda x: abs((x.time().hour * 60 + x.time().minute) - 
                     (target_dt.hour * 60 + target_dt.minute))
    )
    
    # 10分以内の最も近い時刻のデータを取得
    closest = df_ticker[df_ticker['time_diff'] <= 10].nsmallest(1, 'time_diff')
    
    if len(closest) > 0:
        return closest['Close'].iloc[0]
    return None

# 各銘柄の各時刻での価格を取得
results = []

for ticker in grok_tickers:
    row_data = {'ticker': ticker}
    
    # GROK情報を追加
    grok_info = df_grok[df_grok['ticker'] == ticker].iloc[0]
    row_data['stock_name'] = grok_info['stock_name']
    row_data['selection_score'] = grok_info['selection_score']
    row_data['grok_rank'] = grok_info['grok_rank']
    row_data['is_top5'] = grok_info['grok_rank'] <= 5
    row_data['is_top10'] = grok_info['grok_rank'] <= 10
    
    # 寄付価格（日足のOpen）
    row_data['price_09:00'] = get_open_price(ticker)
    
    # 各時刻の売却価格（5分足）
    for time_str in TIME_SLOTS.keys():
        price = get_price_at_time(ticker, time_str)
        row_data[f'price_{time_str}'] = price
    
    results.append(row_data)

df_results = pd.DataFrame(results)

# 寄付価格（買値）を基準に各時刻での利益率を計算
buy_price_col = 'price_09:00'

for time_str in TIME_SLOTS.keys():
    sell_price_col = f'price_{time_str}'
    return_col = f'return_{time_str}'
    
    df_results[return_col] = (
        (df_results[sell_price_col] - df_results[buy_price_col]) / 
        df_results[buy_price_col] * 100
    )

print(f"✅ 利益率計算完了")
print(f"\n寄付価格データがある銘柄数: {df_results[buy_price_col].notna().sum()} / {len(df_results)}")

# データがある銘柄のみにフィルタ
df_results_valid = df_results[df_results[buy_price_col].notna()].copy()

if len(df_results_valid) > 0:
    print(f"\nサンプル（Top5銘柄）:")
    display_cols = ['ticker', 'stock_name', 'grok_rank', buy_price_col] + \
                   [f'return_{t}' for t in TIME_SLOTS.keys()]
    print(df_results_valid[df_results_valid['is_top5']][display_cols].head())
else:
    print("\n⚠️ 分析可能なデータがありません")

✅ 利益率計算完了

寄付価格データがある銘柄数: 12 / 12

サンプル（Top5銘柄）:
   ticker         stock_name  grok_rank  price_09:00  return_09:30  \
1  4592.T              サンバイオ          2       2699.0      2.593553   
3  4417.T  グローバルセキュリティエキスパート          4       4330.0     -4.734411   
4  3911.T             Aiming          1        247.0      0.809717   
6  3936.T           グローバルウェイ          5        162.0      1.851852   
8  7078.T          INCLUSIVE          3        426.0           NaN   

   return_10:00  return_10:30  return_11:00  return_11:30  
1      3.705076      4.001482      4.520193      4.409040  
3     -5.542725     -4.965358     -6.466513     -6.697460  
4      0.404858      0.404858      0.000000      0.404858  
6      0.617284           NaN           NaN           NaN  
8      1.643192           NaN           NaN           NaN  


## 4. 集計統計の計算

In [24]:
# グループ別の統計を計算
def calc_group_stats(df, group_name):
    """グループの統計を計算"""
    stats = {'group': group_name, 'count': len(df)}
    
    for time_str in TIME_SLOTS.keys():
        return_col = f'return_{time_str}'
        
        if return_col in df.columns:
            valid_returns = df[return_col].dropna()
            
            if len(valid_returns) > 0:
                stats[f'{time_str}_avg'] = valid_returns.mean()
                stats[f'{time_str}_win_rate'] = (valid_returns > 0).sum() / len(valid_returns) * 100
                stats[f'{time_str}_count'] = len(valid_returns)
            else:
                stats[f'{time_str}_avg'] = None
                stats[f'{time_str}_win_rate'] = None
                stats[f'{time_str}_count'] = 0
    
    return stats

# 各グループの統計
stats_top5 = calc_group_stats(df_results_valid[df_results_valid['is_top5']], 'Top5')
stats_top10 = calc_group_stats(df_results_valid[df_results_valid['is_top10']], 'Top10')
stats_all = calc_group_stats(df_results_valid, '全銘柄')

df_stats = pd.DataFrame([stats_top5, stats_top10, stats_all])

print("\n📊 グループ別統計:")
print(df_stats[['group', 'count', '11:30_avg', '11:30_win_rate']].to_string(index=False))


📊 グループ別統計:
group  count  11:30_avg  11:30_win_rate
 Top5      5  -0.627854       66.666667
Top10     10  -0.660284       50.000000
  全銘柄     12  -0.780753       40.000000


## 5. ビジュアライゼーション

In [25]:
# グラフ1: 時刻別平均リターン（グループ比較）
time_points = list(TIME_SLOTS.keys())
time_labels = ['9:30', '10:00', '10:30', '11:00', '11:30']

fig_returns = go.Figure()

for idx, row in df_stats.iterrows():
    returns = [row[f'{t}_avg'] for t in time_points]
    
    fig_returns.add_trace(go.Scatter(
        x=time_labels,
        y=returns,
        mode='lines+markers',
        name=row['group'],
        line=dict(width=3),
        marker=dict(size=10)
    ))

fig_returns.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)

fig_returns.update_layout(
    title=f'<b>時刻別平均リターン（{TARGET_DATE}）</b><br><sub>9:00寄付買い → 各時刻売却</sub>',
    xaxis_title='売却時刻',
    yaxis_title='平均リターン (%)',
    height=500,
    hovermode='x unified',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_returns.show()

In [26]:
# グラフ2: 時刻別勝率（グループ比較）
fig_winrate = go.Figure()

for idx, row in df_stats.iterrows():
    win_rates = [row[f'{t}_win_rate'] for t in time_points]
    
    fig_winrate.add_trace(go.Bar(
        x=time_labels,
        y=win_rates,
        name=row['group'],
        text=[f'{v:.1f}%' if v is not None else 'N/A' for v in win_rates],
        textposition='outside'
    ))

fig_winrate.add_hline(y=50, line_dash="dash", line_color="gray", opacity=0.5,
                       annotation_text="50%（損益分岐）")

fig_winrate.update_layout(
    title=f'<b>時刻別勝率（{TARGET_DATE}）</b><br><sub>9:00寄付買い → 各時刻売却</sub>',
    xaxis_title='売却時刻',
    yaxis_title='勝率 (%)',
    height=500,
    barmode='group',
    hovermode='x unified',
    yaxis=dict(range=[0, 100]),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_winrate.show()

In [27]:
# グラフ3: 個別銘柄のリターン（Top5のみ）
df_top5 = df_results_valid[df_results_valid['is_top5']].copy()

if len(df_top5) > 0:
    fig_individual = go.Figure()
    
    for _, row in df_top5.iterrows():
        returns = [row[f'return_{t}'] for t in time_points]
        
        fig_individual.add_trace(go.Scatter(
            x=time_labels,
            y=returns,
            mode='lines+markers',
            name=f"{row['ticker']} {row['stock_name']}",
            hovertemplate='%{y:.2f}%<extra></extra>'
        ))
    
    fig_individual.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    fig_individual.update_layout(
        title=f'<b>Top5銘柄 個別リターン（{TARGET_DATE}）</b><br><sub>9:00寄付買い → 各時刻売却</sub>',
        xaxis_title='売却時刻',
        yaxis_title='リターン (%)',
        height=600,
        hovermode='x unified',
        legend=dict(x=1.02, y=1, bgcolor='rgba(255,255,255,0.8)')
    )
    
    fig_individual.show()
else:
    print("⚠️ Top5銘柄のデータがありません")

## 6. 詳細データテーブル

In [28]:
# Top5銘柄の詳細テーブル
if len(df_top5) > 0:
    print("\n📋 Top5銘柄の詳細:")
    print("="*120)
    
    display_df = df_top5.sort_values('grok_rank')[[
        'grok_rank', 'ticker', 'stock_name', 'selection_score',
        'price_09:00', 'return_09:30', 'return_10:00', 'return_10:30',
        'return_11:00', 'return_11:30'
    ]].copy()
    
    # 数値フォーマット
    for col in display_df.columns:
        if 'return_' in col:
            display_df[col] = display_df[col].apply(lambda x: f"{x:+.2f}%" if pd.notna(x) else "N/A")
        elif col == 'price_09:00':
            display_df[col] = display_df[col].apply(lambda x: f"{x:.0f}" if pd.notna(x) else "N/A")
    
    display_df.columns = [
        'ランク', 'コード', '銘柄名', 'スコア', '寄付',
        '9:30', '10:00', '10:30', '11:00', '11:30(前引)'
    ]
    
    print(display_df.to_string(index=False))
    print("="*120)


📋 Top5銘柄の詳細:
 ランク    コード               銘柄名   スコア   寄付   9:30  10:00  10:30  11:00 11:30(前引)
   1 3911.T            Aiming 148.0  247 +0.81% +0.40% +0.40% +0.00%    +0.40%
   2 4592.T             サンバイオ 160.0 2699 +2.59% +3.71% +4.00% +4.52%    +4.41%
   3 7078.T         INCLUSIVE  88.0  426    N/A +1.64%    N/A    N/A       N/A
   4 4417.T グローバルセキュリティエキスパート 155.0 4330 -4.73% -5.54% -4.97% -6.47%    -6.70%
   5 3936.T          グローバルウェイ  90.0  162 +1.85% +0.62%    N/A    N/A       N/A


## 7. サマリーレポート

In [29]:
print(f"\n" + "="*80)
print(f"📊 GROK銘柄 前場パフォーマンスサマリー ({TARGET_DATE})")
print("="*80)

print(f"\n【戦略】9:00 寄付買い → 前引け（11:30）売却")
print("-"*80)

for _, row in df_stats.iterrows():
    group = row['group']
    count = row['11:30_count']
    avg_return = row['11:30_avg']
    win_rate = row['11:30_win_rate']
    
    print(f"\n{group} ({count}銘柄):")
    if avg_return is not None:
        print(f"  平均リターン: {avg_return:+.2f}%")
        print(f"  勝率: {win_rate:.1f}%")
    else:
        print(f"  データなし")

# 最適売却時刻の特定
print(f"\n\n【最適売却時刻分析】")
print("-"*80)

for group in ['Top5', 'Top10', '全銘柄']:
    group_data = df_stats[df_stats['group'] == group].iloc[0]
    
    best_time = None
    best_return = -999
    
    for t in time_points:
        ret = group_data[f'{t}_avg']
        if ret is not None and ret > best_return:
            best_return = ret
            best_time = t
    
    if best_time:
        time_label = TIME_SLOTS[best_time].replace('売却', '')
        print(f"\n{group}: {time_label} ({best_return:+.2f}%)")
    else:
        print(f"\n{group}: データ不足")

print("\n" + "="*80)


📊 GROK銘柄 前場パフォーマンスサマリー (2025-10-27)

【戦略】9:00 寄付買い → 前引け（11:30）売却
--------------------------------------------------------------------------------

Top5 (3銘柄):
  平均リターン: -0.63%
  勝率: 66.7%

Top10 (4銘柄):
  平均リターン: -0.66%
  勝率: 50.0%

全銘柄 (5銘柄):
  平均リターン: -0.78%
  勝率: 40.0%


【最適売却時刻分析】
--------------------------------------------------------------------------------

Top5: 10:00 (+0.17%)

Top10: 9:30 (+0.12%)

全銘柄: 9:30 (+0.45%)



## 8. HTMLエクスポート（オプション）

グラフをHTMLファイルとして保存してブラウザで開けます。

In [30]:
# HTMLファイルとして保存
output_dir = ROOT / "output" / "grok_analysis"
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

fig_returns.write_html(output_dir / f"returns_{timestamp}.html")
fig_winrate.write_html(output_dir / f"winrate_{timestamp}.html")

if len(df_top5) > 0:
    fig_individual.write_html(output_dir / f"top5_individual_{timestamp}.html")

print(f"\n✅ HTMLファイルを保存しました: {output_dir}")
print(f"\nブラウザで開くには:")
print(f"  open {output_dir}/returns_{timestamp}.html")


✅ HTMLファイルを保存しました: /Users/hiroyukiyamanaka/Desktop/python_stock/dash_plotly/output/grok_analysis

ブラウザで開くには:
  open /Users/hiroyukiyamanaka/Desktop/python_stock/dash_plotly/output/grok_analysis/returns_20251027_223627.html


## 9. Phase2分析: 目標利益率到達判定

**Phase2戦略**: 9:00寄付買い → ⭕%上昇で即売り、11:30前引けまでに未到達なら引け不成

目標利益率: **3%, 5%, 7%**

In [ ]:
from datetime import datetime, time

# Phase2分析: 目標利益率到達判定
PROFIT_TARGETS = [3.0, 5.0, 7.0]  # 3%, 5%, 7%

def detect_profit_target_achievement(ticker, buy_price, target_pct):
    """
    5分足データで目標利益率到達を検知
    
    Args:
        ticker: 銘柄コード
        buy_price: 買値（寄付価格）
        target_pct: 目標利益率（%）
    
    Returns:
        tuple: (到達有無, 到達時刻, 到達価格)
    """
    if pd.isna(buy_price) or buy_price <= 0:
        return False, None, None
    
    # 目標価格を計算
    target_price = buy_price * (1 + target_pct / 100)
    
    # 該当銘柄の5分足データを取得
    df_ticker = df_today_5m[df_today_5m['ticker'] == ticker].copy()
    
    if len(df_ticker) == 0:
        return False, None, None
    
    # 9:00-11:30の範囲でフィルター
    df_ticker = df_ticker[
        (df_ticker['date'].dt.time >= time(9, 0)) &
        (df_ticker['date'].dt.time <= time(11, 30))
    ].sort_values('date')
    
    # 目標価格到達を検知（High価格で判定）
    achieved = df_ticker[df_ticker['High'] >= target_price]
    
    if len(achieved) > 0:
        first_achievement = achieved.iloc[0]
        return True, first_achievement['date'], target_price
    
    return False, None, None


# 各銘柄・各目標利益率で判定
print("Phase2分析: 目標利益率到達判定を実行中...\n")

for target_pct in PROFIT_TARGETS:
    achieved_col = f'target_{int(target_pct)}pct_achieved'
    time_col = f'target_{int(target_pct)}pct_time'
    price_col = f'target_{int(target_pct)}pct_price'
    return_col = f'phase2_return_{int(target_pct)}pct'
    
    results = []
    
    for idx, row in df_results.iterrows():
        ticker = row['ticker']
        buy_price = row['buy_price']
        
        achieved, achievement_time, target_price = detect_profit_target_achievement(
            ticker, buy_price, target_pct
        )
        
        results.append({
            'achieved': achieved,
            'time': achievement_time,
            'price': target_price if achieved else None
        })
    
    # 結果をDataFrameに追加
    df_results[achieved_col] = [r['achieved'] for r in results]
    df_results[time_col] = [r['time'] for r in results]
    df_results[price_col] = [r['price'] for r in results]
    
    # Phase2リターン計算: 到達した場合は目標価格、未到達なら11:30価格
    df_results[return_col] = df_results.apply(
        lambda row: (
            (row[price_col] - row['buy_price']) / row['buy_price'] * 100
            if row[achieved_col] and pd.notna(row[price_col])
            else row['return_11:30']  # 未到達の場合は前引け売却
        ),
        axis=1
    )
    
    achievement_rate = (df_results[achieved_col].sum() / len(df_results)) * 100
    print(f"✓ {target_pct}%目標: {df_results[achieved_col].sum()}/{len(df_results)}銘柄が到達 ({achievement_rate:.1f}%)")

print("\n✅ Phase2分析完了")

## 10. Phase2戦略パフォーマンス集計

In [ ]:
# Phase2戦略の統計を計算
phase2_stats = []

for target_pct in PROFIT_TARGETS:
    return_col = f'phase2_return_{int(target_pct)}pct'
    achieved_col = f'target_{int(target_pct)}pct_achieved'
    
    # 全銘柄統計
    all_avg = df_results[return_col].mean()
    all_win_rate = (df_results[return_col] > 0).sum() / len(df_results) * 100
    all_achievement_rate = (df_results[achieved_col].sum() / len(df_results)) * 100
    
    # Top5統計
    top5_avg = df_results.head(5)[return_col].mean()
    top5_win_rate = (df_results.head(5)[return_col] > 0).sum() / 5 * 100
    top5_achievement_rate = (df_results.head(5)[achieved_col].sum() / 5) * 100
    
    # Top10統計
    top10_avg = df_results.head(10)[return_col].mean()
    top10_win_rate = (df_results.head(10)[return_col] > 0).sum() / 10 * 100
    top10_achievement_rate = (df_results.head(10)[achieved_col].sum() / 10) * 100
    
    phase2_stats.append({
        'target': f'{int(target_pct)}%',
        'all_avg': all_avg,
        'all_win_rate': all_win_rate,
        'all_achievement': all_achievement_rate,
        'top5_avg': top5_avg,
        'top5_win_rate': top5_win_rate,
        'top5_achievement': top5_achievement_rate,
        'top10_avg': top10_avg,
        'top10_win_rate': top10_win_rate,
        'top10_achievement': top10_achievement_rate,
    })

df_phase2_stats = pd.DataFrame(phase2_stats)

print("=" * 80)
print("Phase2戦略パフォーマンス集計")
print("=" * 80)
print(f"\n{'目標':<6} {'グループ':<8} {'平均リターン':<12} {'勝率':<10} {'目標到達率':<10}")
print("-" * 80)

for _, row in df_phase2_stats.iterrows():
    target = row['target']
    print(f"{target:<6} {'All':<8} {row['all_avg']:>10.2f}% {row['all_win_rate']:>8.1f}% {row['all_achievement']:>8.1f}%")
    print(f"{'':6} {'Top5':<8} {row['top5_avg']:>10.2f}% {row['top5_win_rate']:>8.1f}% {row['top5_achievement']:>8.1f}%")
    print(f"{'':6} {'Top10':<8} {row['top10_avg']:>10.2f}% {row['top10_win_rate']:>8.1f}% {row['top10_achievement']:>8.1f}%")
    print()

# Phase1（前引け売却）との比較
print("=" * 80)
print("Phase1 vs Phase2 比較（前引け売却 vs 目標利益売却）")
print("=" * 80)

phase1_all_avg = df_results['return_11:30'].mean()
phase1_top5_avg = df_results.head(5)['return_11:30'].mean()
phase1_top10_avg = df_results.head(10)['return_11:30'].mean()

print(f"\n{'戦略':<20} {'All平均':<12} {'Top5平均':<12} {'Top10平均':<12}")
print("-" * 80)
print(f"{'Phase1(11:30売却)':<20} {phase1_all_avg:>10.2f}% {phase1_top5_avg:>10.2f}% {phase1_top10_avg:>10.2f}%")

for _, row in df_phase2_stats.iterrows():
    strategy_name = f"Phase2({row['target']}目標)"
    print(f"{strategy_name:<20} {row['all_avg']:>10.2f}% {row['top5_avg']:>10.2f}% {row['top10_avg']:>10.2f}%")

print("=" * 80)

## 11. Phase2可視化: 目標到達率と平均リターン

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Phase2可視化: 3つのチャートを作成
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Phase2目標到達率（グループ別）',
        'Phase2平均リターン（グループ別）',
        'Phase1 vs Phase2比較（Top5）',
        'Phase1 vs Phase2比較（Top10）'
    ),
    specs=[
        [{'type': 'bar'}, {'type': 'bar'}],
        [{'type': 'bar'}, {'type': 'bar'}]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

# 1. 目標到達率（グループ別）
for group, color in [('all', '#636EFA'), ('top5', '#EF553B'), ('top10', '#00CC96')]:
    fig.add_trace(
        go.Bar(
            x=df_phase2_stats['target'],
            y=df_phase2_stats[f'{group}_achievement'],
            name=group.upper(),
            marker_color=color,
            text=df_phase2_stats[f'{group}_achievement'].round(1),
            textposition='outside',
            texttemplate='%{text}%'
        ),
        row=1, col=1
    )

# 2. 平均リターン（グループ別）
for group, color in [('all', '#636EFA'), ('top5', '#EF553B'), ('top10', '#00CC96')]:
    fig.add_trace(
        go.Bar(
            x=df_phase2_stats['target'],
            y=df_phase2_stats[f'{group}_avg'],
            name=group.upper(),
            marker_color=color,
            text=df_phase2_stats[f'{group}_avg'].round(2),
            textposition='outside',
            texttemplate='%{text}%',
            showlegend=False
        ),
        row=1, col=2
    )

# 3. Phase1 vs Phase2比較（Top5）
comparison_top5 = {
    'Phase1(11:30)': phase1_top5_avg,
    'Phase2(3%)': df_phase2_stats.iloc[0]['top5_avg'],
    'Phase2(5%)': df_phase2_stats.iloc[1]['top5_avg'],
    'Phase2(7%)': df_phase2_stats.iloc[2]['top5_avg'],
}

fig.add_trace(
    go.Bar(
        x=list(comparison_top5.keys()),
        y=list(comparison_top5.values()),
        marker_color=['#AB63FA', '#FFA15A', '#19D3F3', '#FF6692'],
        text=[f'{v:.2f}%' for v in comparison_top5.values()],
        textposition='outside',
        showlegend=False
    ),
    row=2, col=1
)

# 4. Phase1 vs Phase2比較（Top10）
comparison_top10 = {
    'Phase1(11:30)': phase1_top10_avg,
    'Phase2(3%)': df_phase2_stats.iloc[0]['top10_avg'],
    'Phase2(5%)': df_phase2_stats.iloc[1]['top10_avg'],
    'Phase2(7%)': df_phase2_stats.iloc[2]['top10_avg'],
}

fig.add_trace(
    go.Bar(
        x=list(comparison_top10.keys()),
        y=list(comparison_top10.values()),
        marker_color=['#AB63FA', '#FFA15A', '#19D3F3', '#FF6692'],
        text=[f'{v:.2f}%' for v in comparison_top10.values()],
        textposition='outside',
        showlegend=False
    ),
    row=2, col=2
)

# レイアウト調整
fig.update_xaxes(title_text="目標利益率", row=1, col=1)
fig.update_yaxes(title_text="到達率(%)", row=1, col=1)

fig.update_xaxes(title_text="目標利益率", row=1, col=2)
fig.update_yaxes(title_text="平均リターン(%)", row=1, col=2)

fig.update_xaxes(title_text="戦略", row=2, col=1)
fig.update_yaxes(title_text="平均リターン(%)", row=2, col=1)

fig.update_xaxes(title_text="戦略", row=2, col=2)
fig.update_yaxes(title_text="平均リターン(%)", row=2, col=2)

fig.update_layout(
    height=900,
    title_text=f"Phase2分析: 目標利益売却戦略の検証 ({target_date})",
    showlegend=True,
    legend=dict(x=0.02, y=0.98)
)

fig.show()

# HTML保存
html_file = PARQUET_DIR.parent / "notebooks" / f"phase2_analysis_{target_date.replace('-', '')}.html"
fig.write_html(str(html_file))
print(f"\n✅ Phase2分析グラフを保存: {html_file}")

## 12. Phase2詳細: 銘柄別の目標到達状況

In [ ]:
# Top10銘柄の目標到達状況を可視化
df_top10_detail = df_results.head(10).copy()

# 表示用カラムを選択
display_cols = ['ticker', 'company_name', 'buy_price']

for target_pct in PROFIT_TARGETS:
    achieved_col = f'target_{int(target_pct)}pct_achieved'
    time_col = f'target_{int(target_pct)}pct_time'
    return_col = f'phase2_return_{int(target_pct)}pct'
    
    display_cols.extend([achieved_col, time_col, return_col])

df_display = df_top10_detail[display_cols].copy()

# カラム名をわかりやすく変更
rename_dict = {'ticker': '銘柄コード', 'company_name': '企業名', 'buy_price': '買値'}
for target_pct in PROFIT_TARGETS:
    target_str = f'{int(target_pct)}%'
    rename_dict[f'target_{int(target_pct)}pct_achieved'] = f'{target_str}到達'
    rename_dict[f'target_{int(target_pct)}pct_time'] = f'{target_str}到達時刻'
    rename_dict[f'phase2_return_{int(target_pct)}pct'] = f'{target_str}リターン'

df_display = df_display.rename(columns=rename_dict)

# 到達時刻を文字列に変換（時:分のみ）
for target_pct in PROFIT_TARGETS:
    target_str = f'{int(target_pct)}%'
    time_col = f'{target_str}到達時刻'
    df_display[time_col] = df_display[time_col].apply(
        lambda x: x.strftime('%H:%M') if pd.notna(x) else '-'
    )

# 到達フラグを記号に変換
for target_pct in PROFIT_TARGETS:
    target_str = f'{int(target_pct)}%'
    achieved_col = f'{target_str}到達'
    df_display[achieved_col] = df_display[achieved_col].apply(lambda x: '✓' if x else '✗')

print("=" * 120)
print("Top10銘柄のPhase2目標到達状況")
print("=" * 120)
print(df_display.to_string(index=False))
print("=" * 120)

# 成功パターン抽出（3%以上到達した銘柄）
successful_stocks = df_results[df_results['target_3pct_achieved'] == True].copy()

if len(successful_stocks) > 0:
    print(f"\n✅ 3%目標到達銘柄: {len(successful_stocks)}銘柄")
    print("-" * 80)
    
    for idx, stock in successful_stocks.head(10).iterrows():
        ticker = stock['ticker']
        name = stock['company_name']
        achievement_time = stock['target_3pct_time'].strftime('%H:%M') if pd.notna(stock['target_3pct_time']) else '-'
        return_3 = stock['phase2_return_3pct']
        
        # 5%、7%も到達したか
        reached_5 = '✓' if stock['target_5pct_achieved'] else '✗'
        reached_7 = '✓' if stock['target_7pct_achieved'] else '✗'
        
        print(f"{ticker} {name[:15]:<15} | 3%到達: {achievement_time} ({return_3:>5.2f}%) | 5%: {reached_5} | 7%: {reached_7}")
else:
    print("\n⚠️  本日は3%目標到達銘柄がありませんでした")

print("=" * 120)